In [ ]:
%pip install scikit-learn 
%pip install pandas
%pip install lightgbm
%pip install plotly
%pip install tqdm
%pip install statsmodels

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error
from tqdm import tqdm

                                                    Loading and Pre-processing of Data

In [80]:
# Load in the data
train_df = pd.read_csv('./train.csv', index_col='id')
test_df = pd.read_csv('./test.csv', index_col='id')
stores_df = pd.read_csv('./stores.csv')
transactions_df = pd.read_csv('./transactions.csv')
oil_df = pd.read_csv('./oil.csv')
holidays_events_df = pd.read_csv('./holidays_events.csv')

In [81]:
########## train, test, and 'X' ##########

train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])
# Filter the DataFrame based on 'date'
train_df = train_df[train_df['date'] >= '2016-08-15']

# split train_df into 'y' for target and 'train_features_df' for features from train set.
y = np.log1p(train_df['sales'])
train_features_df = train_df.drop(['sales'], axis=1)
# Create 'X' total features dataframe by combining train and test feature dataframes, so I can feature engineer both train and test at same time.
X = pd.concat([train_features_df, test_df], axis=0, ignore_index=True)

# I decided to not (one hot encode) 'family', but I could have.
le = LabelEncoder()
X['family'] = le.fit_transform(X['family'])

In [82]:
########## stores_df ##########

# Population, size (km^2), and population density (P/km^2) for the cities 
population = {'Quito': 2000000, 'Santo Domingo': 460000, 'Cayambe': 40000, 'Latacunga': 100000, 'Riobamba': 157000, 'Ibarra': 150000, 'Guaranda': 35000, 'Puyo': 40000, 'Ambato': 350000, 'Guayaquil': 2750000, 'Salinas': 50000, 'Daule': 130000, 'Babahoyo': 105000, 'Quevedo': 200000, 'Playas': 40000, 'Libertad': 105000, 'Cuenca': 445000, 'Loja': 200000, 'Machala': 260000, 'Esmeraldas': 200000, 'Manta': 240000, 'El Carmen': 120000}

size = {'Quito': 372, 'Santo Domingo': 60, 'Cayambe': 378, 'Latacunga': 370, 'Riobamba': 59, 'Ibarra': 242, 'Guaranda': 520, 'Puyo': 88, 'Ambato': 47, 'Guayaquil': 345, 'Salinas': 27, 'Daule': 475, 'Babahoyo': 175, 'Quevedo': 300, 'Playas': 280, 'Libertad': 28, 'Cuenca': 71, 'Loja': 44, 'Machala': 67, 'Esmeraldas': 70, 'Manta': 60, 'El Carmen': 1250}

population_density = {'Quito': 5376, 'Santo Domingo': 7667, 'Cayambe': 106, 'Latacunga': 270, 'Riobamba': 2659, 'Ibarra': 620, 'Guaranda': 67, 'Puyo': 454,'Ambato': 7447, 'Guayaquil': 7971, 'Salinas': 1852, 'Daule': 274, 'Babahoyo': 600, 'Quevedo': 667, 'Playas': 143, 'Libertad': 3750, 'Cuenca': 6268, 
'Loja': 4545, 'Machala': 3881, 'Esmeraldas': 2857, 'Manta': 4000, 'El Carmen': 96}

# Map the city to its population & population density
stores_df['city_pop'] = stores_df['city'].map(population)
stores_df['city_pop_density'] = stores_df['city'].map(population_density)

# Manually ordinal encoding into ranks.
def rank(population, population_density):
    if population < 100000:
        pop_rank = 0
    elif population < 300000:
        pop_rank = 1
    elif population < 1000000:
        pop_rank = 2
    else:
        pop_rank = 3
        
    if population_density < 250:
        pop_density_rank = 0
    elif population_density < 800:
        pop_density_rank = 1
    else:
        pop_density_rank = 2
        
    return pop_rank, pop_density_rank

# Create new useful feature columns from 'rank' function.
stores_df[['city_pop', 'city_pop_density']] = stores_df.apply(lambda row: pd.Series(rank(row['city_pop'], row['city_pop_density'])), axis=1)
stores_df['city'] = le.fit_transform(stores_df['city'])
stores_df['type'] = le.fit_transform(stores_df['type'])
stores_drops = ['state', 'cluster']
stores_df = stores_df.drop(stores_drops, axis=1)

In [83]:
########## transactions_df ##########

# Too hard for me & beginners, would have to predict future transactions separately then add as a feature, don't feel like doing that. (We can't just use the transactions data to train our model, because the transactions for 08-16 to 08-31 are not available yet!) (Also doesn't really make sense to do a lag of a day or a few days for transactions, I don't think that would be a useful lag indicator or helpful for predicting the current day sales from previous days transactions.)

########## oil_df ##########

# I think in the spirit of this competition, I can probably use the SMA of the oil prices, and if I wanted to be strict I could have lagged them by 1-3 days or something. It makes sense that oil price lags would still be useful to predict the current days sales.

# Convert date, fill in NaN entries with neighbor values.
oil_df['date'] = pd.to_datetime(oil_df['date'])
# Create a complete date range from the start to the end of your current dates, converts to df and merges to original.
all_dates = pd.date_range(start=oil_df['date'].min(), end=oil_df['date'].max())
all_dates_df = pd.DataFrame(all_dates, columns=['date'])
oil_df = pd.merge(all_dates_df, oil_df, on='date', how='left')
# backfill missing values, create moving average.
oil_df['oil_price'].fillna(method='bfill', inplace=True)
oil_df['oil_price_sma_7'] = oil_df['oil_price'].rolling(window=7).mean().round(2)
# drop the original oil price because sma is better and more usable.
oil_df = oil_df.drop('oil_price', axis=1)

In [84]:
########## holidays_events_df ########## (What a convoluted mess!)

# Convert date, renames 'type' column to 'holiday_type' for readability
holidays_events_df['date'] = pd.to_datetime(holidays_events_df['date'])
holidays_events_df = holidays_events_df.rename(columns={'type': 'holiday_type'})
# Within 'transferred' column, 'True' just means it's no longer a holiday on that day, so we can keep only rows where it's False.
holidays_events_df = holidays_events_df[holidays_events_df['transferred'] == False]
# Within 'holiday_type' reclass redundant information as simply 'Holiday'.
replacements = ['Transfer', 'Bridge', 'Additional']
holidays_events_df['holiday_type'] = holidays_events_df['holiday_type'].replace(replacements, 'Holiday')
# Next, keeping only 'National' holidays for 'locale', that's good enough for me.
holidays_events_df = holidays_events_df[~holidays_events_df['locale'].isin(['Local', 'Regional'])]
# Keep only the first occurrence of each date
holidays_events_df = holidays_events_df.drop_duplicates(subset='date')
# Drop the useless or too granular columns, and reset index. You could one hot encode or maybe label encode, but idc dropping is fine.
holiday_drops = ['locale', 'locale_name', 'description', 'transferred']
holidays_events_df = holidays_events_df.drop(holiday_drops, axis=1)
holidays_events_df = holidays_events_df.reset_index(drop=True)

# 3. Merge the polished versions of the data into our Features dataframe 'X'.
X = X.merge(stores_df, on ='store_nbr', how='left')
X = X.merge(oil_df, on='date', how='left')
X = X.merge(holidays_events_df, on='date', how='left')

In [85]:
# Fill NaN values with a string 'None'
X['holiday_type'] = X['holiday_type'].fillna('None')
# one-hot encoding on 'holiday_type'
X = pd.get_dummies(X, columns=['holiday_type'])

# Rename one-hot encoded columns
renamed_columns = X.columns.str.replace('holiday_type_', '')
X.columns = renamed_columns
# Creating more features
X['day'] = X['date'].dt.day
X['month'] = X['date'].dt.month
X['year'] = X['date'].dt.year
X['day_of_week'] = X['date'].dt.dayofweek
X['is_weekend'] = X['day_of_week'].isin([5, 6]).astype(int) 
X['payday'] = ((X['date'].dt.day == 15) | (X['date'].dt.is_month_end)).astype(int)

# splitting back into training and test sets
X_train = X[X['date'] < '2017-08-16']
X_test = X[X['date'] >= '2017-08-16']
# resets indexes
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

X_train = X_train.drop(columns='date')
X_test = X_test.drop(columns='date')

In [92]:
# X
# 678,942 rows, all good, with date column
# y
# 650,430 rows all good, just 'sales'.
# X_train
# 650,430 rows all good, no date column.
# X_test
# 28,512 rows all good, no date column.
X_train

,store_nbr,family,onpromotion,city,type,city_pop,city_pop_density,oil_price_sma_7,Event,Holiday,None,Work Day,day,month,year,day_of_week,is_weekend,payday
0,1,0,0,18,3,3,2,44.24,0,0,1,0,15,8,2016,0,0,1
1,1,1,0,18,3,3,2,44.24,0,0,1,0,15,8,2016,0,0,1
2,1,2,0,18,3,3,2,44.24,0,0,1,0,15,8,2016,0,0,1
3,1,3,37,18,3,3,2,44.24,0,0,1,0,15,8,2016,0,0,1
4,1,4,0,18,3,3,2,44.24,0,0,1,0,15,8,2016,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650425,9,28,0,18,1,3,2,48.18,0,0,1,0,15,8,2017,1,0,1
650426,9,29,1,18,1,3,2,48.18,0,0,1,0,15,8,2017,1,0,1
650427,9,30,148,18,1,3,2,48.18,0,0,1,0,15,8,2017,1,0,1
650428,9,31,8,18,1,3,2,48.18,0,0,1,0,15,8,2017,1,0,1


                                                            Multiple models looping - Fit

In [63]:
# Extract the unique combinations of store_nbr and family
store_family_pairs = [tuple(x) for x in X_train[['store_nbr', 'family']].drop_duplicates().values]

# Initialize an empty dictionary to store the models
models = {}

# Loop through each pair
for pair in tqdm(store_family_pairs):
    # Filter the dataframe for the current pair
    X_train_pair = X_train[(X_train['store_nbr'] == pair[0]) & (X_train['family'] == pair[1])]
    y_pair = y[X_train_pair.index]
    
    # Initialize and train a model for the current pair
    lgbm = LGBMRegressor(learning_rate = 0.01, n_estimators = 300, num_leaves = 80, n_jobs=2)
    lgbm.fit(X_train_pair, y_pair)
    
    # Store the model in the dictionary using the pair as the key
    models[pair] = lgbm

    # 0.441065 = lr 0.02, nest=250, num leaves= 50

100%|██████████| 1782/1782 [01:10<00:00, 25.40it/s]


                                                        Multiple models looping - Prediction on Train

In [64]:
# Initialize an empty list to store the predictions
y_pred_train = []
indices = []

# Loop through each model and make predictions
for pair, model in models.items():
    # Filter the dataframe for the current pair
    X_train_pair = X_train[((X_train['store_nbr'] == pair[0]) & (X_train['family'] == pair[1]))]
    
    # Make predictions and append them to the list
    y_pred_train.extend(model.predict(X_train_pair))

    # Store the indices
    indices.extend(X_train_pair.index.tolist())

# Create a DataFrame from the predictions and indices
predictions_df = pd.DataFrame({'Index': indices, 'Prediction': y_pred_train})

# Sort the predictions DataFrame by index
predictions_df.sort_values(by='Index', inplace=True)

# Convert the predictions and truth to the original scale
y_pred_train = np.maximum(predictions_df['Prediction'].values, 0)
y_pred_train = np.expm1(y_pred_train)
y_train_expm1 = np.expm1(y)

# Calculate the RMSLE on the training data
rmsle_train = np.sqrt(mean_squared_log_error(y_train_expm1, y_pred_train))
print(f"RMSLE on Training data: {rmsle_train}")

RMSLE on Training data: 0.3886955484472582


                                                    Multiple models looping - Prediction on Test

In [65]:
# Initialize an empty list to store the predictions
y_pred_test = []
indices = []

# Loop through each model and make predictions
for pair, model in models.items():
    # Filter the dataframe for the current pair
    X_test_pair = X_test[( (X_test['store_nbr'] == pair[0]) & (X_test['family'] == pair[1]) )]
    
    # Make predictions and append them to the list
    y_pred_test.extend(model.predict(X_test_pair))

    # Store the indices
    indices.extend(X_test_pair.index.tolist())

# Create a DataFrame from the predictions and indices
predictions_df = pd.DataFrame({'Index': indices, 'Prediction': y_pred_test})

# Sort the predictions DataFrame by index
predictions_df.sort_values(by='Index', inplace=True)

# Convert the predictions to the original scale and clip them at 0
y_pred_test = np.maximum(predictions_df['Prediction'].values, 0)
y_pred_test = np.expm1(y_pred_test)

In [66]:
df_submission = pd.DataFrame()
df_submission['id'] = range(len(y_pred_test))
df_submission['id'] += 3000888
df_submission['sales'] = y_pred_test

# Reorder columns
df_submission = df_submission[['id', 'sales']]
df_submission.to_csv('submission4.csv', index=False)

In [108]:
correlation_values = X_train.corr(method='spearman').values
feature_names = X_train.columns.values

data = [
    go.Heatmap(
        z=correlation_values,
        x=feature_names,
        y=feature_names,
        colorscale='Jet',
        reversescale=False,
        text=correlation_values,
        opacity=1.0
    )
]

layout = go.Layout(
    title='Spearman Correlation of Features',
    xaxis=dict(ticks='', nticks=36),
    yaxis=dict(ticks=''),
    width=900,
    height=700
)

fig = go.Figure(data=data, layout=layout)
fig.show()